This is continution of [part1 notebook](https://github.com/JpChii/ML-Projects/blob/main/Spacy.ipynb)

# Processing Pipelines

Learn everything needed about spaCy's processing pipeline. Learn what goes uner the hood when processing a text and to write custom components and add them to pipeline, custom attributes to add own metadata to the documents, spans and tokens.

![alt text](https://course.spacy.io/pipeline.png "What happens when nlp is called?")

**Built-in pipeline components**

|Name|Description|Creates|
|----|-----------|-------|
| tagger | Part-of-speech tagger | Token.tag, Token.pos |
| parser | Dependency parser | Token.dep, Token.head, Doc.sents, Doc.noun_chunks |
| ner | Named entity recogonizer | Doc.ents, Token.ent_iob, Token.ent_type |
| textcat | Text classifier | Doc.cats |

**Under the hood**
![alt text](https://course.spacy.io/package_meta.png "What happens when nlp is called?")

* Pipeline defined in model's `config.cfg` in order
* Built-inn components need binary data to make predictions

### 1.1 Inspecting the pipeline


In [8]:
import spacy
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [7]:
import spacy

nlp = spacy.load("en_core_web_sm")

print(nlp.pipeline)
print(nlp.pipe_names)

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x7f78b8376750>), ('tagger', <spacy.pipeline.tagger.Tagger object at 0x7f78b8376980>), ('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x7f78bc6593d0>), ('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x7f78bc6fea50>), ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer object at 0x7f78bc6ffd70>), ('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x7f78bc6594d0>)]
['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


## 2. Custom Pipeline components

**why custom components?**

* Make a function execute automatically when calling nlp
* Add own metadata to documents and tokens
* Updating built-in attributes like `doc.ents`

### 2.1 Simple component

In [5]:
import spacy
from spacy.language import Language

# Define the custom component
@Language.component("length_component")
def length_component_function(doc):
    # Get the doc's length
    doc_length = len(doc)
    print(f"This document is {doc_length} tokens long.")
    # Return the doc
    return doc


# Load the small English pipeline
nlp = spacy.load("en_core_web_sm")

# Add the component first in the pipeline and print the pipe names
nlp.add_pipe("length_component", first=True)
print(nlp.pipe_names)

# Process a text
doc = nlp("Tst in")

['length_component', 'tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
This document is 2 tokens long.


### 2.2 Complex components

Write a custom component that uses the `PhraseMatcher` to find animal names in the document and adds the matched spans to the doc.ents

In [9]:
import spacy
from spacy.language import Language
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

nlp = spacy.load("en_core_web_sm")
animals = ["Golden Retriever", "cat", "turtle", "Rattus norvegicus"]
animal_patterns = list(nlp.pipe(animals))

print("animal_patterns", animal_patterns)
matcher = PhraseMatcher(nlp.vocab)
animal_patterns = list(nlp.pipe(animals))
matcher.add("ANIMAL", animal_patterns)

# Define the custom component
@Language.component("animal_component")
def animal_component_function(doc):
    # Apply matcher to the doc
    matches = matcher(doc)
    # Create a Span for each match and assign the label "ANIMAL"
    spans = [Span(doc, start, end, label="ANIMAL") for match_id, start, end in matches]
    # Overwrite the doc.ents with the matched spans
    doc.ents = spans
    
    return doc

# Add the component to the pipeline after the "ner" component
nlp.add_pipe("animal_component", after="ner")
print(nlp.pipe_names)

# Process the text and print the text and label for the doc.ents
doc = nlp("I have a cat and a Golden Retriever")
print([(ent.text, ent.label_) for ent in doc.ents])

animal_patterns [Golden Retriever, cat, turtle, Rattus norvegicus]
['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner', 'animal_component']
[('cat', 'ANIMAL'), ('Golden Retriever', 'ANIMAL')]


Yay!! Written a custom pipeline component for rule-based entity matching.

## 3. Extension attributes

* Add custom metadata to documents, tokens and spans
* Accessible via the ._ property

**Extension attributes types**

1. Attribute extensions
2. Property extensions
3. Method extensions

### 3.1 Setting extension attributes(1)

In [14]:
# Attribute extension
import spacy
from spacy.tokens import Token

nlp = spacy.blank("en")

# Register the Token extension attribute "is_country" with the default value False
Token.set_extension("is_country", default=False, force=True)

# Process the text and set the is_country attribute to True for the token "Spain"
doc = nlp("I live in Spain.")
doc[3]._.is_country = True

# Print the token text and the is_country attribute for all tokens
print([(token.text, token._.is_country) for token in doc])

[('I', False), ('live', False), ('in', False), ('Spain', True), ('.', False)]


In [23]:
# Getter function extension
import spacy
from spacy.tokens import Token

nlp = spacy.blank("en")

# Defining the getter function
def get_reversed(token):
    return token.text[::-1]

Token.set_extension("reversed", getter=get_reversed)

doc = nlp("All generalizations are false, including this one.")
for token in doc:
    print(f"reversed: {token._.reversed}")

reversed: llA
reversed: snoitazilareneg
reversed: era
reversed: eslaf
reversed: ,
reversed: gnidulcni
reversed: siht
reversed: eno
reversed: .


### 3.2 Setting extension attributes (2)

Set an extension function for the `Doc`

In [36]:
import spacy
from spacy.tokens import Doc

nlp = spacy.blank("en")

# Define the getter function
def get_has_number(doc):
    # Return if any of the tokens in the doc return True for token.like_num
    return any(token.like_num for token in doc)

Doc.set_extension("has_number", method=get_has_number, force=True)

# Process the text and check the custom has_number attribute
doc = nlp("The museum closed for five years in 2012.")
print(f"has_number: {doc._.has_number}")

has_number: functools.partial(<function get_has_number at 0x7f78bb1585f0>, The museum closed for five years in 2012.)


In [34]:
import spacy
from spacy.tokens import Span

nlp = spacy.blank("en")

# Define the method
def to_html(span, tag):
    # Wrap the span text in a HTML tag and return it
    return f"<{tag}>{span.text}</{tag}>"


# Register the Span method extension "to_html" with the method to_html
Span.set_extension("to_html", method=to_html, force=True)

# Process the text and call the to_html method on the span with the tag name "strong"
doc = nlp("Hello world, this is a sentence.")
span = doc[0:2]
print(span._.to_html("strong"))

<strong>Hello world</strong>


### 3.3 Entities and extensions

Getting a wikipedia url if the span's label is in the list of labels

In [49]:
import spacy
from spacy.tokens import Span

nlp = spacy.load("en_core_web_sm")

def get_wikipedia_url(span):
    if span.label_ in ("PERSON", "ORG", "GPE", "LOCATION"):
        entity_text = span.text.replace(" ", "_")
        return "https://en.wikipedia.org/w/index.php?search=" + entity_text

Span.set_extension("wiki_url", getter=get_wikipedia_url, force=True)

doc = nlp(
    "In over fifty years from his very first recordings right through to his "
    "last album, David Bowie was at the vanguard of contemporary culture."
)

for ent in doc.ents:
    print(ent.text, ent._.wiki_url)

over fifty years None
first None
David Bowie https://en.wikipedia.org/w/index.php?search=David-Bowie


Yay!! Created a pipeline component that uses named entities predicted by the model to generate wikipedia URLs and adds them as a custom attribute

In [50]:
nlp=spacy.blank("en")
nlp.pipe_names

[]

In [ ]:
import json
import spacy
from spacy.language import Language
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher

with open("exercises/en/countries.json", encoding="utf8") as f:
    COUNTRIES = json.loads(f.read())

with open("exercises/en/capitals.json", encoding="utf8") as f:
    CAPITALS = json.loads(f.read())

nlp = spacy.blank("en")
matcher = PhraseMatcher(nlp.vocab)
matcher.add("COUNTRY", list(nlp.pipe(COUNTRIES)))


@Language.component("countries_component")
def countries_component_function(doc):
    # Create an entity Span with the label "GPE" for all matches
    matches = matcher(doc)
    doc.ents = [Span(doc, start, end, label="GPE") for match_id, start, end in matches]
    return doc


# Add the component to the pipeline
nlp.add_pipe("countries_component", first=True)
print(nlp.pipe_names)

# Getter that looks up the span text in the dictionary of country capitals
get_capital = lambda span: CAPITALS.get(span.text)

# Register the Span extension attribute "capital" with the getter get_capital
Span.set_extension("captial", getter=get_capital)

# Process the text and print the entity text, label and capital attributes
doc = nlp("Czech Republic may help Slovakia protect its airspace")
print([(ent.text, ent.label_, ent._.capital) for ent in doc.ents])

This is an example of how to add structured data to a spaCy pipeline.

## 4. Scaling and Performance

**Processing large volumes of text**

* Use `nlp.pipe` method
* Preprocesses texts as a stream, yields `Doc` objects
* Much faster than calling `nlp` on each text

**Passing in context(1)**

* Setting `as_tuples=True` on `nlp.pipe` lets you pass in `(text, context)` tuples
* Yields `(doc, context)` tuples
* Useful for associating metadata with `doc`

### 4.1 Processing streams

In [51]:
import json
import spacy

with open("data/tweets.json", encoding="utf8") as f:
    TEXTS = json.loads(f.read())

In [61]:
# Getting all the tweet contents alone
tweets = [tweet["content"] for tweet in TEXTS]
tweets[0]

'Be sure to tune in and watch Donald Trump on Late Night with David Letterman as he presents the Top Ten List tonight!'

In [67]:
# Process the texts and print the adjectives
for doc in nlp.pipe(tweets[:20]):
    print([token.text for token in doc if token.pos_ == "ADJ"])

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [ ]:
import json
import spacy

nlp = spacy.load("en_core_web_sm")

with open("exercises/en/tweets.json", encoding="utf8") as f:
    TEXTS = json.loads(f.read())

# Process the texts and print the entities
# docs = [nlp(text) for text in TEXTS] # Bad performance
docs = list(nlp.pipe(docs)) # Good perforance
entities = [doc.ents for doc in docs]
print(*entities)

### 4.3 Preprocessing with context

In this exercise, you’ll be using custom attributes to add author and book meta information to quotes.

A list of [text, context] examples is available as the variable DATA. The texts are quotes from famous books, and the contexts dictionaries with the keys "author" and "book".

Use the set_extension method to register the custom attributes "author" and "book" on the Doc, which default to None.
Process the [text, context] pairs in DATA using nlp.pipe with as_tuples=True.
Overwrite the doc._.book and doc._.author with the respective info passed in as the context.

In [ ]:
import json
import spacy
from spacy.tokens import Doc

with open("exercises/en/bookquotes.json", encoding="utf8") as f:
    DATA = json.loads(f.read())

nlp = spacy.blank("en")

# Register the Doc extension "author" (default None)
Doc.set_extension("author", default=None)

# Register the Doc extension "book" (default None)
Doc.set_extension("book", default=None)

for doc, context in nlp.pipe(DATA, as_tuples=True):
    # Set the doc._.book and doc._.author attributes from the context
    doc._.book = context["book"]
    doc._.author = context["author"]

    # Print the text and custom attribute data
    print(f"{doc.text}\n — '{doc._.book}' by {doc._.author}\n")

### 4.4 Selecective processing

In this excercise, you'll use the `nlp.make_doc` and `nlp.select_pipes` method to run only selected components when running a text.

In [70]:
import spacy

nlp = spacy.load("en_core_web_sm")
text = (
    "Chick-fil-A is an American fast food restaurant chain headquartered in "
    "the city of College Park, Georgia, specializing in chicken sandwiches."
)

# Performing only toenization
doc = nlp.make_doc(text)
print([token.text for token in doc])

['Chick', '-', 'fil', '-', 'A', 'is', 'an', 'American', 'fast', 'food', 'restaurant', 'chain', 'headquartered', 'in', 'the', 'city', 'of', 'College', 'Park', ',', 'Georgia', ',', 'specializing', 'in', 'chicken', 'sandwiches', '.']


In [71]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [72]:
# Running only the ner disabling tagger and lemmetizer
with nlp.select_pipes(enable="ner"):
    doc = nlp(doc)
    print(doc.ents)

(American, College Park, Georgia)
